<a href="https://colab.research.google.com/github/simrathanspal/deep_models_from_scratch/blob/main/IP_P2_Text_Classification_Representative_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Representation model - Task specific



In [19]:
! pip install datasets transformers sentence-transformers openai evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00


# Loading Dataset

In [2]:
from datasets import load_dataset
data = load_dataset("rotten_tomatoes")
data

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [5]:
data['train'][0,-1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

# Using Pretrained task specific model (BERT with pretrained classifier)

In [10]:
from transformers import pipeline

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# Pipeline type is inferred from the model
classification_pipeline = pipeline(model=model_path,
                                   tokenizer=model_path,
                                   return_all_scores=True,
                                   device="cuda:0"
                                  )


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return

In [11]:
classification_pipeline('the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .')

[[{'label': 'negative', 'score': 0.00460449093952775},
  {'label': 'neutral', 'score': 0.0880218893289566},
  {'label': 'positive', 'score': 0.9073736071586609}]]

In [12]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred = []
for output in tqdm(classification_pipeline(KeyDataset(data["test"], "text")), total=len(data["test"])):
  negative_score = output[0]["score"]
  positive_score = output[2]["score"]
  assignment = np.argmax([negative_score, positive_score])
  y_pred.append(assignment)

100%|██████████| 1066/1066 [00:10<00:00, 101.46it/s]


In [ ]:
data["test"]["text"]

Column(['lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .', 'consistently clever and suspenseful .', 'it\'s like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .', 'the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .', 'red dragon " never cuts corners .'])

In [13]:
from sklearn.metrics import classification_report


def evaluate_performance(y_true, y_pred):
  performance = classification_report(y_true, y_pred, target_names=["negative", "positive"])
  print(performance)

evaluate_performance(data["test"]["label"], y_pred)

              precision    recall  f1-score   support

    negative       0.76      0.88      0.81       533
    positive       0.86      0.72      0.78       533

    accuracy                           0.80      1066
   macro avg       0.81      0.80      0.80      1066
weighted avg       0.81      0.80      0.80      1066



In [ ]:
print(negative_score, positive_score, assignment)

0.8362504243850708 0.012839589267969131 0


In [14]:
from sklearn.metrics import classification_report


def evaluate_performance(y_true, y_pred):
  performance = classification_report(y_true, y_pred, target_names=["negative", "positive"])
  print(performance)

evaluate_performance(data["test"]["label"], y_pred)

              precision    recall  f1-score   support

    negative       0.76      0.88      0.81       533
    positive       0.86      0.72      0.78       533

    accuracy                           0.80      1066
   macro avg       0.81      0.80      0.80      1066
weighted avg       0.81      0.80      0.80      1066



# Representation - Task specific (Fine tuned)

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
text = 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'
inputs = tokenizer(text, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  1996,  2600,  2003, 16036,  2000,  2022,  1996,  7398,  2301,
          1005,  1055,  2047,  1000, 16608,  1000,  1998,  2008,  2002,  1005,
          1055,  2183,  2000,  2191,  1037, 17624,  2130,  3618,  2084,  7779,
         29058,  8625, 13327,  1010,  3744,  1011, 18856, 19513,  3158,  5477,
          4168,  2030,  7112, 16562,  2140,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [4]:
def preprocess(examples):
  return tokenizer(examples["text"], truncation=True, padding="max_length")

train_data = data["train"].map(preprocess, batched=True)
test_data = data["test"].map(preprocess, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [5]:
# Trainable parameters

num_labels = data["train"].features["label"].num_classes
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Freeze all parameters of the BERT model
for param in model.base_model.parameters():
  param.requires_grad = False

# A Pooler layer is a Linear Layer that takes the input of [CLS] = 768 dim input
# We are making this layer trainable
# Empirical results show using [CLS] embedding directly works much better than pooled embedding
# Pooled embedding was used for Next Setence Prediction (NSP)

# (pooler): BertPooler(
#       (dense): Linear(in_features=768, out_features=768, bias=True)
#       (activation): Tanh()

# Option 1 using CLS
# outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
# last_hidden_state = outputs.last_hidden_state
# cls_output = last_hidden_state[:, 0] # all batch, first output
# cls_output = self.dropout(cls_output)
# logits = self.classifier(cls_output)

# Option 2 using Pooled output
# outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
# cls_output = outputs.pooler_output
# cls_output = self.dropout(cls_output)
# logits = self.classifier(cls_output)

# The author of https://www.youtube.com/watch?v=EAIil0wD-1A turned on pooler layer
# But pooler layer was specifically trained for predicting IsNext and NotNext
# Hence the performance of Pooler layer is not great for classification task
# [CLS] will perform better for classification task because it summarizes the document.
# for param in model.bert.pooler.parameters():
#   param.requires_grad = True

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
help(list(model.parameters())[0].numel)

Help on built-in function numel:

numel(...) method of torch.nn.parameter.Parameter instance
    numel() -> int

    See :func:`torch.numel`



In [15]:
def count_parameters(model):
  total_params = sum(p.numel() for p in model.parameters())
  trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

  return {
      "Total": total_params,
      "Trainable": trainable_params,
      "Frozen": total_params - trainable_params
  }

param_counts = count_parameters(model)
print("Model Parameter Counts:")
for k, v in param_counts.items():
  print(f"{k}: {v:,}")

Model Parameter Counts:
Total: 109,483,778
Trainable: 1,538
Frozen: 109,482,240


This makes sence 768*2 + 2

In [16]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [17]:
# Before finetuning
model(**inputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3173, -0.3841]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [21]:
import evaluate
import numpy as np
from scipy.special import softmax


# Load the evaluation metric
accuracy_metric = evaluate.load("accuracy")

# Define compute_metrics
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  # If we only want to know which class won - argmax is fine
  # But, if we want to know confidence we need softmax at inference
  # During training softmax is absolutely necessary because we are comparing
  # probability distribution

  # Stable Softmax
  # Keepdims = True to keep the batch wise max
  max_logits = np.max(logits, axis=-1, keepdims=True)
  stable_logits = logits - max_logits

  logits = np.exp(logits)
  predictions = np.exp(stable_logits)/np.sum(np.exp(stable_logits), axis=-1, keepdims=True)
  predictions = np.argmax(logits, axis=-1)
  return accuracy_metric.compute(predictions=predictions, references=labels)

In [22]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to = "none"
)

In [23]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = test_data,
    compute_metrics = compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.687200,0.673200,0.558161
2,0.696400,0.660781,0.593809
3,0.638000,0.652278,0.614447
4,0.645800,0.643317,0.651970


# Representation model - Embedding based

In [ ]:
! pip install datasets transformers sentence-transformers openai

In [ ]:
from datasets import load_dataset
data = load_dataset("rotten_tomatoes")
data

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
train_embeddings = model.encode(data["train"]["text"], show_progress_bar = True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar = True)

train_embeddings.shape

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

(8530, 768)

In [ ]:
from sklearn.metrics import classification_report


def evaluate_performance(y_true, y_pred):
  performance = classification_report(y_true, y_pred, target_names=["negative", "positive"])
  print(performance)

evaluate_performance(data["test"]["label"], y_pred)

              precision    recall  f1-score   support

    negative       0.85      0.86      0.85       533
    positive       0.86      0.85      0.85       533

    accuracy                           0.85      1066
   macro avg       0.85      0.85      0.85      1066
weighted avg       0.85      0.85      0.85      1066



In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

              precision    recall  f1-score   support

    negative       0.85      0.86      0.85       533
    positive       0.86      0.85      0.85       533

    accuracy                           0.85      1066
   macro avg       0.85      0.85      0.85      1066
weighted avg       0.85      0.85      0.85      1066

